# Import Instaloader

In [1]:
import shutil
import sqlite3
from glob import glob
from os.path import expanduser
from instaloader import Instaloader, ConnectionException, Post
from instaloader.exceptions import QueryReturnedBadRequestException, TooManyRequestsException

# Authenticate your burner account on instagram

In [2]:
import instaloader

L = instaloader.Instaloader()
L.interactive_login("sentiments_detection")

Enter Instagram password for sentiments_detection:  ········


# Load and inject cookies for authentication

In [3]:
# Step 1: Locate and copy Firefox cookies
path_to_cookies = expanduser("~/Library/Application Support/Firefox/Profiles/*/cookies.sqlite")
original_cookie_file = glob(path_to_cookies)[0]
shutil.copy(original_cookie_file, "cookies_temp.sqlite")

# Step 2: Load Instagram cookies
conn = sqlite3.connect("cookies_temp.sqlite")
cursor = conn.cursor()
cookies = dict(cursor.execute("SELECT name, value FROM moz_cookies WHERE host='.instagram.com'"))

# Step 3: Inject cookies into Instaloader session
L = Instaloader(max_connection_attempts=1)
L.context._session.cookies.update(cookies)

# Step 4: Test and save session
try:
    username = L.test_login()
    if not username:
        raise ConnectionException()
except ConnectionException:
    raise SystemExit("Cookie import failed. Are you logged in successfully in Firefox?")

L.context.username = username
L.save_session_to_file()

Saved session to /Users/avanimahawar/.config/instaloader/session-sentiments_detection.


In [4]:
print(L.test_login())  # should print your username

sentiments_detection


# Insert sortcode to fetch comments data 

In [9]:
import instaloader
import time

COMMENTS_TO_FETCH = 25
RETRY_DELAY = 5  # seconds
MAX_RETRIES = 3

L = instaloader.Instaloader()
L.load_session_from_file('sentiments_detection')  # after successful login

SHORTCODES = 'DLJb_caouAE,DLxdqQfsvMk'
shortcode_list = [code.strip() for code in SHORTCODES.split(',')]

count = 0
comments_data = [] # List to hold comments data

def fetch_comments(post, max_comments=COMMENTS_TO_FETCH):
    count = 0
    for comment in post.get_comments():
        retries = 0
        while retries < MAX_RETRIES:
            try:
                # Append comment info to list
                comments_data.append({
                    'post_shortcode': post.shortcode,
                    'username': comment.owner.username,
                    'comment': comment.text
                })
                count += 1
                time.sleep(1)  # Safe delay between requests

                if count >= max_comments:
                    return count
                break  # Exit retry loop if successful

            except instaloader.exceptions.TooManyRequestsException:
                print("Rate limit hit. Sleeping for 15 minutes...")
                time.sleep(15 * 60)

            except instaloader.exceptions.QueryReturnedBadRequestException:
                print("Bad request — possibly throttled. Sleeping for 5 minutes...")
                time.sleep(5 * 60)

            except instaloader.exceptions.ConnectionException as e:
                print(f"Connection error: {e}. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
                retries += 1

            except Exception as e:
                print(f"Unexpected error: {e}. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
                retries += 1

        else:
            print(f"[WARN] Skipping comment after {MAX_RETRIES} failed attempts.")

    return count

# Iterate over all shortcodes
total_comments = 0
for shortcode in shortcode_list:
    print(f"\nFetching comments for post: {shortcode}")
    post = instaloader.Post.from_shortcode(L.context, shortcode)
    total_comments += fetch_comments(post)
    
print(f"\nTotal comments fetched from all posts: {total_comments}")

Loaded session from /Users/avanimahawar/.config/instaloader/session-sentiments_detection.

Fetching comments for post: DLJb_caouAE


JSON Query to graphql/query: 403 Forbidden when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]



Fetching comments for post: DLxdqQfsvMk


JSON Query to graphql/query: 403 Forbidden when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]



Total comments fetched from all posts: 14


# Data Cleaning and Translation

In [10]:
import pandas as pd
df_comments = pd.DataFrame(comments_data)

In [27]:
df_comments

,post_shortcode,username,comment
0,DLJb_caouAE,siddhartha.banerjee.35110,🔥🔥😍
1,DLJb_caouAE,debendra_saha_07,Ai TMC sokar hindu birodhi
2,DLJb_caouAE,roy_ankit_2000,Mamata Banerjee was not allowed in the Jaganna...
3,DLJb_caouAE,rajgupta3171,Mamta ji is destroyed hindu bangali
4,DLJb_caouAE,rajnishmsd._007,😢
5,DLJb_caouAE,itanumay,@cpimwb
6,DLJb_caouAE,ak_ku.48,G@@ndu gola to abar giye mamtaz kei vote debe ...
7,DLxdqQfsvMk,tali7165,An accent for every occasion 🙄
8,DLxdqQfsvMk,dr.lansyhomeandgarden,abominal
9,DLxdqQfsvMk,lafartstudio,KEEP THIS MAN OUT OF OFFICE


In [28]:
from googletrans import Translator

translator = Translator()

for comment in comments_data:
    try:
        translated = translator.translate(comment['comment'], src='auto', dest='en')
        comment['comment_en'] = translated.text
    except Exception as e:
        print(f"Translation failed for comment: {comment['comment']}. Error: {e}")
        comment['comment_en'] = comment['comment']  # fallback

In [29]:
# Convert to DataFrame
df_comments = pd.DataFrame(comments_data)

# Optional: reorder columns
df_comments = df_comments[['post_shortcode', 'username', 'comment', 'comment_en']]

df_comments

,post_shortcode,username,comment,comment_en
0,DLJb_caouAE,siddhartha.banerjee.35110,🔥🔥😍,🔥🔥😍
1,DLJb_caouAE,debendra_saha_07,Ai TMC sokar hindu birodhi,This Tomke is anti -Hindu
2,DLJb_caouAE,roy_ankit_2000,Mamata Banerjee was not allowed in the Jaganna...,Mamata Banerjee was not allowed in the Jaganna...
3,DLJb_caouAE,rajgupta3171,Mamta ji is destroyed hindu bangali,Mamta ji this Destroid Hindu Bengali
4,DLJb_caouAE,rajnishmsd._007,😢,😢
5,DLJb_caouAE,itanumay,@cpimwb,@cp a
6,DLJb_caouAE,ak_ku.48,G@@ndu gola to abar giye mamtaz kei vote debe ...,G @@ Nadu Gola will go again and Mumtaz will v...
7,DLxdqQfsvMk,tali7165,An accent for every occasion 🙄,An accent for every occasion 🙄
8,DLxdqQfsvMk,dr.lansyhomeandgarden,abominal,abominal
9,DLxdqQfsvMk,lafartstudio,KEEP THIS MAN OUT OF OFFICE,KEEP THIS MAN OUT OF OFFICE


# Write data to google sheets

In [12]:
!pip install gspread gspread_dataframe oauth2client

In [30]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

In [31]:
# Google API scope
scope = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]

In [32]:
# Authenticate
creds = ServiceAccountCredentials.from_json_keyfile_name("sentiments-469614-b3ccd856e4a4.json", scope)
client = gspread.authorize(creds)

In [33]:
# Open Google Sheet by name (or use .open_by_key(sheet_id))
sheet = client.open("raw_data").sheet1

# Clear old data
sheet.clear()

# Write df_comments to Google Sheet
set_with_dataframe(sheet, df_comments)

print("DataFrame uploaded successfully!")

DataFrame uploaded successfully!
